# Generating random Tweets

This notebook uses Markov Chains models in order to generate random texts (with a maximum length equal to a Tweet).

Although it can generate reasonably accurate results, it's a completely synthetic dataset! Because of this, instead of using the generated data as training data for our NER model, we simply use it to study which text operation can improve its accuracy.

## 1. Generating Markov Chains model

Here we simply provide a series of raw text files in order to create our model - which we then save as a JSON file.

In [1]:
import markovify
import glob

max_tweet_size = 140
raw_text_files_path = 'data/raw/markov_text_files/*.txt'
parsed_model_file_path = 'data/parsed/markov_text_files/markov_weighted_chain.json'
raw_text_file_paths = glob.glob(raw_text_files_path)
raw_text_markov_models = []

# Read raw text files and generate Markov chain models
for file_path in raw_text_file_paths:
    with open(file_path) as file:
        text = file.read()
        markov_model = markovify.Text(text, state_size=4)
        raw_text_markov_models.append(markov_model)
    
# Combine all generated models into a single one
markov_model = markovify.combine(raw_text_markov_models)

In [2]:
# Generate 5 random sentences from the generated Markov chain model
for i in range(5):
    print(markov_model.make_short_sentence(max_tweet_size))

_In deeply seated bones_, such as the upper end of the humerus, may be the seat of excruciating pain.
I had no idea that he was in fear of someone or something which drove him from America.
He swore on his honor that the Russians were running away and that he himself was formulating.
Quinine, in 5 to 10 grain doses every four hours, have also been employed with apparent benefit.
He looked about him anxiously in the glare of the streets could at first see nothing.


In [3]:
# Save model as JSON
model_json = markov_model.chain.to_json()
with open(parsed_model_file_path, 'w') as json_file:
    json_file.write(model_json)

## 2. Improving the model

Even though we want to generate completely random Tweet-like texts, our aim is to improve the accuracy of our NER model for Country/Nationality/Religion/Currency recognition. Having said that, we will go through our model, generate a set amount of samples and use them as testing data.

In [4]:
import pandas as pd

# Define target datasets' paths
parsed_country_nationality_file = 'data/parsed/parsed_country_nationality.csv'
parsed_currency_country_file = 'data/parsed/parsed_currency_country.csv'
parsed_country_religion_file = 'data/parsed/country_religion_files/parsed_country_religion.csv'
parsed_country_cities_file = 'data/parsed/parsed_country_cities.csv'

# Load the necessary datasets
country_nationality_df = pd.read_csv(parsed_country_nationality_file, encoding='utf-8', compression='gzip', index_col=False)
currency_country_df = pd.read_csv(parsed_currency_country_file, encoding='utf-8', compression='gzip', index_col=False)
country_religion_df = pd.read_csv(parsed_country_religion_file, encoding='utf-8', compression='gzip', index_col=False)
country_cities_df = pd.read_csv(parsed_country_cities_file, encoding='utf-8', compression='gzip', index_col=False)

# Store unique sets
unique_country_common_names = country_nationality_df['Common Name'].astype(str).unique()
unique_country_official_names = country_nationality_df['Official Name'].astype(str).unique()
unique_country_nationalities = country_nationality_df['Nationality'].astype(str).unique()
unique_country_religions_name = country_religion_df['Religion'].astype(str).unique()
unique_country_rilogions_affiliation = country_religion_df['Affiliation'].astype(str).unique()
unique_currency_ids = currency_country_df['ID'].astype(str).unique()
unique_country_city_names = country_cities_df['City'].astype(str).unique()
# TODO also add currencies' full names

In [5]:
import re
import random

# Define important word sets
important_word_sets = [
    list(map(lambda x : x.upper(), unique_country_common_names)),
    list(map(lambda x : x.upper(), unique_country_official_names)),
    list(map(lambda x : x.upper(), unique_country_nationalities)),
    list(map(lambda x : x.upper(), unique_country_religions_name)),
    list(map(lambda x : x.upper(), unique_country_rilogions_affiliation)),
    list(map(lambda x : x.upper(), unique_currency_ids)),
    list(map(lambda x : x.upper(), unique_country_city_names))
]

def is_word_important(word):
    '''
    This method checks wether or not a word
    is considered to be 'important'.
    '''
    # Set regex for word parsing
    regex = re.compile('[^a-zA-Z]')
    
    for important_word_set in important_word_sets:
        comparable_word = regex.sub('', word).upper()
        if comparable_word in important_word_set:
            return True
    return False

In [6]:
def generate_testing_samples_dict(max_testing_samples, unnecessary_sample_retention_percent):
    # Define number of sentences to generate
    cur_testing_samples = 0

    testing_sample_dict = {}
    while len(testing_sample_dict.keys()) < max_testing_samples:
        batch_size = max_testing_samples - cur_testing_samples
        samples = [markov_model.make_short_sentence(max_tweet_size) for i in range(batch_size)]
        
        for sample in samples:
            contains_important_word = False
            if sample is None:
                continue
            for word in sample.split():
                if is_word_important(word):
                    contains_important_word = True
                    break
            if (contains_important_word):
                print('\n[New Sample]\nText: {}'.format(sample))
                n_city_names = int(input('[1/6] # City Names: '))
                n_country_names = int(input('[2/6] # Country Names: '))
                n_country_nationalities = int(input('[3/6] # Nationalities: '))
                n_religion_names = int(input('[4/6] # Religion names: '))
                n_religion_affiliations = int(input('[5/6] # Religious affiliations: '))
                n_currency_names = int(input('[6/6] # Currency names: '))
                
                n_param_sum = n_city_names + n_country_names + n_country_nationalities + n_religion_names + n_religion_affiliations + n_currency_names
                if (n_param_sum > 0 or random.randint(0,100) <= unnecessary_sample_retention_percent):
                    testing_sample_dict[cur_testing_samples] = {
                        'Text': sample,
                        'City Names': n_city_names,
                        'Country Names': n_country_names,
                        'Country Nationalities': n_country_nationalities,
                        'Religion Names': n_religion_names,
                        'Religion Affiliations': n_religion_affiliations,
                        'Currency Names': n_currency_names
                    }
                    cur_testing_samples += 1
                    print('[{}/{} SAVED]'.format(cur_testing_samples, max_testing_samples))
                else:
                    print('[DISCARDED]')
    return testing_sample_dict

In [7]:
sample_dict = generate_testing_samples_dict(2, 20)


[New Sample]
Text: Pacific Ocean The Pacific Ocean is a major contributor to the world economy has made manufacturing in China much more cost effective.
[1/5] # Country Names: 0
[2/5] # Nationalities: 0
[3/5] # Religion names: 0
[4/5] # Religious affiliations: 0
[5/5] # Currency names: 0
[DISCARDED]

[New Sample]
Text: They all appear to be adhesive, and there is a remarkable absence of pain.
[1/5] # Country Names: 0
[2/5] # Nationalities: 0
[3/5] # Religion names: 0
[4/5] # Religious affiliations: 0
[5/5] # Currency names: 0
[DISCARDED]

[New Sample]
Text: Having cleared the way Denisov stopped at the end of the Cold War allowed for German unification in 1990.
[1/5] # Country Names: 0
[2/5] # Nationalities: 0
[3/5] # Religion names: 0
[4/5] # Religious affiliations: 0
[5/5] # Currency names: 0
[DISCARDED]

[New Sample]
Text: The ejaculation had been drawn from my companion by the fact that there was nothing left to do as a Russian.
[1/5] # Country Names: 0
[2/5] # Nationalities: 0
[3

In [8]:
sample_df = pd.DataFrame.from_dict(sample_dict, orient='index')
sample_df.head()

,Text,Country Names,Country Nationalities,Religion Names,Religion Affiliations,Currency Names
0,Moldova's dependence on Russian energy was und...,1,0,1,1,1
1,China generally has implemented reforms in a g...,0,0,0,0,0
